In [ ]:
import numpy as np
#worst case for calculate Loss
def hinge_loss(X, y, w):
    """
    Compute Hinge Loss for multi-class classification.
    
    :param X: Input data (n, d)
    :param y: True labels (n,)
    :param w: Weights (d, c)
    :return: Scalar loss value
    """
    n, c = X.shape[0], w.shape[1]
    
    # Compute the scores (raw model outputs)
    scores = np.dot(X, w)  # (n, c)
    
    # Compute Hinge Loss
    loss = 0
    for i in range(1000):
        correct_class_score = scores[i, y[i]]  # Score for the correct class
        for j in range(c):
            if j != y[i]:  # For all classes except the correct one
                loss += max(0, 1 - correct_class_score + scores[i, j])  # Hinge Loss

    # Average the loss
    loss /= n
    return loss

# Random initialization of data
np.random.seed(0)
n_samples = 10000
n_features = 3073
n_classes = 10

X_train = np.random.randn(n_samples, n_features)  # Input data
y_train = np.random.randint(0, n_classes, size=n_samples)  # Random labels

# Searching for weights with the lowest loss
best_loss = float("inf")
best_W = None

for i in range(10000):
    w = np.random.randn(n_features, n_classes) * 0.0001  # Random initialization of weights
    loss = hinge_loss(X_train, y_train, w)  # Compute the loss
    if loss < best_loss:
        best_loss = loss
        best_W = w
    print("In attempt %d the loss was %f, best %f" % (i, loss, best_loss))


In [ ]:
# Multi calss SVM Loss function

def L_i_vectorized(x, y, W):
    scores = W.dot(x)
    margins = np.maximum(0, scores - scores[y] + 1)
    margins[y] = 0
    loss_i = np.sum(margins)
    return loss_i

In [ ]:
import numpy as np

def eval_numerical_gradient(f, x):
    """
    A function to compute the numerical gradient of a function f at the point x using finite differences.
    
    :param f: The function to compute the gradient for.
    :param x: The point (numpy array) at which the gradient is computed.
    :return: The gradient (numpy array) at the point x.
    """
    fx = f(x)  # Compute the value of the function at x
    grad = np.zeros_like(x)  # Initialize the gradient array (same shape as x)
    h = 0.00001  # Small step size for finite difference

    # Use np.nditer to iterate over all elements of x (supporting multi-dimensional arrays)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        ix = it.multi_index  # Get the index of the current element
        old_value = x[ix]  # Save the original value of x[ix]
        
        # Evaluate f(x + h) by adding a small step to the current element
        x[ix] += h
        fxh = f(x)  # Compute the function value at x + h
        
        # Restore the original value of x[ix]
        x[ix] = old_value

        # Compute the partial derivative using the finite difference method
        grad[ix] = (fxh - fx) / h
        
        it.iternext()  # Move to the next element

    return grad


#Numerical gradient: Approximate, time-consuming, easy to implement!
#Analytical gradient: Accurate, fast, prone to implementation errors!

